In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/Datasets/kaggle_rock_new.csv" "kaggle_rock_new.csv"

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tqdm.auto import tqdm
import string
import pickle
from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
SEED = 42
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
DATASET = "./kaggle_rock_new.csv"
df = pd.read_csv(DATASET)
del df['Unnamed: 0']
df

,lyrics
0,a lot of cats are hatin' slandering makin' bad...
1,somebody tell me why we landed here on the pla...
2,i'm spittin' with the venom\nto your soul thro...
3,where should i begin cripplin' all you villain...
4,enough of all that let's switch up the format\...
...,...
100252,break down we've got to make them see\nno disc...
100253,everything comes to a question where time is t...
100254,you got to climb up on your high horses decide...
100255,it all comes tumbling down\nno vital parts rem...


In [ ]:
def sanitize(text):
    out = ''
    #  placing spaces around each line separator to separate them from the words
    #  removing all chars that are neither letters nor spaces
    for c in text:
        if c == '':
            out += " \n "
        elif c not in string.ascii_letters and c != ' ':
            out += ''
        else:
            out += c
        
    return out

words = df['lyrics'].astype('string').dropna()\
    .apply(sanitize).astype('string').dropna()\
    .apply(str.split)\
    .explode()
words

0               a
0             lot
0              of
0            cats
0             are
           ...   
100256    getting
100256         us
100256       down
100256       down
100256       down
Name: lyrics, Length: 19681337, dtype: object

In [ ]:
counts = words.value_counts()
# get all words that are quite rare to exclude them from the vocabulary
threshold = 30
to_remove = counts[counts <= threshold]
# words.replace(to_remove, np.nan, inplace=True)
# words = words.dropna()
# words
to_remove

jealousies     30
nectar         30
arun           30
fixin          30
hazard         30
               ..
deprecate       1
promisary       1
scryer          1
manipulater     1
ibyeoreul       1
Name: lyrics, Length: 100228, dtype: int64

In [ ]:
unique = pd.unique(words)
# form a vocabulary
vocab, index = {}, 3
vocab['<pad>'] = 0
vocab[''] = 1
vocab['\n'] = 2
for token in unique:
    if token not in to_remove:
        vocab[token] = index
        index += 1
vocab_size = len(vocab)

In [ ]:
vocab_size

12653

In [ ]:
inverse_vocab = {index : token for token, index in vocab.items()}
inverse_vocab

In [ ]:
def generate_training_data(df, vocab, inverse_vocab, blacklist, window_size, num_ns, vocab_size, seed):
    targets = []
    
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)
    bad_skip_gram = False
    for key, lyrics in tqdm(df["lyrics"].items()):
        lyrics = sanitize(lyrics)
        tokens = lyrics.split(' ')
        tokens = filter(lambda t: t not in blacklist and t in vocab, tokens)
        vectorized_lyrics = [vocab[word] for word in tokens]
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
                vectorized_lyrics,
                vocabulary_size=vocab_size,
                window_size=window_size,
                negative_samples=0)
        
        for target_word, context_word in positive_skip_grams:
            targets.append(target_word)
    return targets

In [ ]:
keys, values = zip(*vocab.items())

In [ ]:
output = open('my_dict.pkl', 'wb')
pickle.dump(vocab, output)
output.close()

In [ ]:
vocab_file = open("dictionary.txt", "w")
vocab_file.write("\n".join([str(values[i]) + '\n' + str(keys[i]) for i in range(len(keys))]))

158195

In [ ]:
output_file = open("./drive/My Drive/LeakGAN/output_file.txt", "a")

In [ ]:
def add_newLines(x):
  if x == 2:
    return '\n'
  else:
    return str(x)

index = 0;
step = 500;
while index + step < df.shape[0]:
  training_data = generate_training_data(df[index : index + step], vocab, inverse_vocab, to_remove,
                       5, 4, vocab_size, SEED)
  modified_list = " ".join(list(map(add_newLines, training_data)))
  output_file.write("".join([s for s in modified_list.strip().splitlines(True) if s.strip()]));
  index = index + step
  # saving the constructure of rows but skipping the empty lines between songs  